# Ensemble Learning

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
df = pd.read_csv('all_predictions.csv')
df = df.set_index('date')
df.head()

,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Sentiments,logistic_c_with_sentiment,logistic_c_without_sentiment,linear_r_with_sentiment,linear_r_without_sentiment,BNN_r_without_sentiment,...,LSTM_r_without_sentiment,LSTM_r_with_sentiment,LSTM_c_without_sentiment,LSTM_c_with_sentiment,XGBoost_r_without_sentiment,XGBoost_r_with_sentiment,XGBoost_c_without_sentiment,XGBoost_c_with_sentiment,VAR_r_without_sentiment,VAR_r_with_sentiment
date,,,,,,,,,,,,,,,,,,,,,
1/1/21,29374.1523,28994.0098,29600.6270,28803.5859,positive,1,1,31206.99161,31692.53211,39634.580,...,31882.857,30772.920,1,1,31541.07,30630.19,1,1,26537.86,26559.36
2/1/21,32127.2676,29376.4551,33155.1172,29091.1816,positive,1,1,29038.13669,29899.26833,39646.926,...,31817.668,31171.412,1,1,32369.48,32020.37,1,1,26872.01,26791.62
3/1/21,32782.0234,32129.4082,34608.5586,32052.3164,positive,1,1,30654.52027,29856.58873,39638.160,...,33739.656,32062.006,1,1,32259.98,32218.10,1,1,25132.14,25326.21
4/1/21,31971.9141,32810.9492,33440.2188,28722.7559,positive,1,1,30839.92107,30069.32171,39634.580,...,34207.027,32406.158,1,1,32295.86,32593.87,1,0,25342.24,26057.93
5/1/21,33992.4297,31977.0410,34437.5898,30221.1875,positive,1,1,33570.39941,33860.22146,39634.580,...,33240.855,31999.209,1,1,33103.70,33381.27,1,1,27093.09,28210.31


In [3]:
df['class_y'] = df["Adj_Close_BTC-USD"].shift(1).dropna()
df['class_y'] = df.apply(lambda x: 1 if x["class_y"] < x["Adj_Close_BTC-USD"] else 0 , axis = 1)

## With Sentiments

### Regression

In [4]:
df_senti = df[['linear_r_with_sentiment', 'BNN_r_with_sentiment', 'LSTM_r_with_sentiment', 'XGBoost_r_with_sentiment','VAR_r_with_sentiment']]
df_senti['pred'] = df_senti.mean(axis = 1)
train_rmse = np.sqrt(mean_squared_error(df['Adj_Close_BTC-USD'].loc[:"14/3/21"], df_senti['pred'].loc[:"14/3/21"]))
test_rmse = np.sqrt(mean_squared_error(df['Adj_Close_BTC-USD'].loc["29/3/21":], df_senti['pred'].loc["29/3/21":]))
print(train_rmse, test_rmse)

3156.552891717284 1763.710313426157


C:\Users\Samantha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Classification

In [5]:
df_senti_c = df[['logistic_c_with_sentiment', 'BNN_c_with_sentiment', 'LSTM_c_with_sentiment', 'XGBoost_c_with_sentiment']]
df_senti_c['pred'] = round(df_senti_c.mean(axis = 1))
train_acc = accuracy_score(df['class_y'].loc[:"14/3/21"], df_senti_c['pred'].loc[:"14/3/21"])
test_acc = accuracy_score(df['class_y'].loc["29/3/21":], df_senti_c['pred'].loc["29/3/21":])
print(train_acc, test_acc)

0.8493150684931506 0.8


C:\Users\Samantha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Without Sentiments

### Regression

In [6]:
df_n_senti = df[['linear_r_without_sentiment', 'BNN_r_without_sentiment', 'LSTM_r_without_sentiment', 
                 'XGBoost_r_without_sentiment','VAR_r_without_sentiment']]
df_n_senti['pred'] = df_n_senti.mean(axis = 1)
train_rmse = np.sqrt(mean_squared_error(df['Adj_Close_BTC-USD'].loc[:"14/3/21"], df_n_senti['pred'].loc[:"14/3/21"]))
test_rmse = np.sqrt(mean_squared_error(df['Adj_Close_BTC-USD'].loc["29/3/21":], df_n_senti['pred'].loc["29/3/21":]))
print(train_rmse, test_rmse)

2591.3552075151265 2851.75993812135


C:\Users\Samantha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Classification

In [7]:
df_n_senti_c = df[['logistic_c_without_sentiment', 'BNN_c_without_sentiment', 
                   'LSTM_c_without_sentiment', 'XGBoost_c_without_sentiment']]
df_n_senti_c['pred'] = round(df_n_senti_c.mean(axis = 1))
train_acc = accuracy_score(df['class_y'].loc[:"14/3/21"], df_n_senti_c['pred'].loc[:"14/3/21"])
test_acc = accuracy_score(df['class_y'].loc["29/3/21":], df_n_senti_c['pred'].loc["29/3/21":])
print(train_acc, test_acc)

0.7534246575342466 0.6


C:\Users\Samantha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
